In [55]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import config
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

In [78]:
data = pd.read_csv("datasets/cleaned_crime_data.csv")
data

,ID,Case Number,Date,Primary Type,Description,Arrest,Domestic,District,Ward,Community Area,...,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count,generalized_loc,Lat_round,Lon_round,location_crime_count
0,13311263,JG503434,2022-07-29 03:39:00,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,True,False,10.0,25.0,30.0,...,7,NaN,NaN,9676,224691,20979,Residential,NaN,NaN,NaN
1,13053066,JG103252,2023-01-03 16:44:00,NARCOTICS,MANUFACTURE / DELIVER - CRACK,True,False,11.0,28.0,26.0,...,1,NaN,NaN,31555,246748,19935,Street/Outdoor,NaN,NaN,NaN
2,13203321,JG415333,2023-09-06 17:00:00,CRIMINAL DAMAGE,TO VEHICLE,False,False,1.0,42.0,32.0,...,9,41.886018,-87.633938,141930,246748,21305,Residential,41.886,-87.634,278.0
3,13204489,JG416325,2023-09-06 11:00:00,THEFT,OVER $500,False,False,1.0,4.0,32.0,...,9,41.871835,-87.626151,268613,246748,21305,Residential,41.872,-87.626,141.0
4,12419690,JE295655,2021-07-07 10:30:00,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,False,False,5.0,10.0,54.0,...,7,41.655116,-87.594883,6241,195093,18040,Residential,41.655,-87.595,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160662,13802159,JJ212459,2025-04-09 22:30:00,ASSAULT,SIMPLE,False,False,2.0,4.0,38.0,...,4,41.805127,-87.608894,110690,54103,4949,Street/Outdoor,41.805,-87.609,79.0
1160663,13801723,JJ213223,2025-04-09 16:45:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,...,4,41.893923,-87.639581,90187,54103,4949,Residential,41.894,-87.640,25.0
1160664,13801213,JJ212469,2025-04-09 23:00:00,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,False,True,15.0,37.0,25.0,...,4,41.891120,-87.762457,110690,54103,4949,Residential,41.891,-87.762,13.0
1160665,13806474,JJ218891,2025-03-31 04:00:00,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,18.0,42.0,8.0,...,3,41.893180,-87.634686,90187,54103,17655,Street/Outdoor,41.893,-87.635,234.0


In [90]:
data = data.dropna(ignore_index=False, subset=["Arrest", "Primary Type", "District", "Latitude", "Longitude"])
data = data[data["Year"] <= 2024]


Feature Selection

1. Use Label Encoder to handle categorical attributes
2. Use standard z-score to normalize data


In [91]:
# Encode categorical attributes
le = LabelEncoder()
data["Primary Type"] = le.fit_transform(data["Primary Type"])
data["generalized_loc"] = le.fit_transform(data["generalized_loc"])
data["Arrest"] = data["Arrest"].astype(int)
data["Domestic"] = data["Domestic"].astype(int)
data

,ID,Case Number,Date,Primary Type,Description,Arrest,Domestic,District,Ward,Community Area,...,Month,Latitude,Longitude,primary_type_count,total_crimes_per_yr,monthly_crime_count,generalized_loc,Lat_round,Lon_round,location_crime_count
2,13203321,JG415333,2023-09-06 17:00:00,5,TO VEHICLE,0,0,1.0,42.0,32.0,...,9,41.886018,-87.633938,141930,246748,21305,5,41.886,-87.634,278.0
3,13204489,JG416325,2023-09-06 11:00:00,22,OVER $500,0,0,1.0,4.0,32.0,...,9,41.871835,-87.626151,268613,246748,21305,5,41.872,-87.626,141.0
4,12419690,JE295655,2021-07-07 10:30:00,20,SEXUAL EXPLOITATION OF A CHILD,0,0,5.0,10.0,54.0,...,7,41.655116,-87.594883,6241,195093,18040,5,41.655,-87.595,65.0
5,12729745,JF279458,2022-06-14 14:47:00,19,ATTEMPT STRONG ARM - NO WEAPON,1,0,16.0,30.0,15.0,...,6,41.945232,-87.766735,46438,224691,19472,1,41.945,-87.767,61.0
6,12835559,JF406130,2022-09-21 22:00:00,14,AUTOMOBILE,1,0,3.0,6.0,69.0,...,9,41.769347,-87.615008,97077,224691,21207,3,41.769,-87.615,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160266,13805547,JJ217844,2024-10-16 12:00:00,8,FINANCIAL IDENTITY THEFT OVER $ 300,0,0,25.0,30.0,19.0,...,10,41.926521,-87.769726,90187,240458,20926,5,41.927,-87.770,18.0
1160368,13805658,JJ218028,2024-12-01 00:00:00,8,FINANCIAL IDENTITY THEFT OVER $ 300,0,0,19.0,47.0,6.0,...,12,41.955088,-87.669072,90187,240458,18073,3,41.955,-87.669,80.0
1160413,13805913,JJ217745,2024-10-22 14:50:00,8,FRAUD OR CONFIDENCE GAME,0,0,19.0,44.0,6.0,...,10,41.936778,-87.648107,90187,240458,20926,1,41.937,-87.648,239.0
1160441,13805552,JJ217956,2024-11-15 12:00:00,8,FORGERY,0,0,2.0,3.0,38.0,...,11,41.819360,-87.615176,90187,240458,18317,5,41.819,-87.615,30.0


Train for each model

In [92]:
# Function to make sure that the folder exists or else we would make a new one
def ensure_path(path):
    if not os.path.exists(path):
        os.mkdir(path)


In [93]:
# Create a map of the model name and the model

# Configurations for Random Forest, DecisionTree, and XGBoost. I set the configs to be similar so we can better compare the results of those models
MAX_DEPTH = 20
MAX_LEAF = 20
IMPURITY = 0.5 #A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
LEARNING_RATE = 0.0001

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "KNNClassifier": KNeighborsClassifier(n_neighbors=20, metric="cityblock"),
    "DecisionTreeClassifier": DecisionTreeClassifier(criterion="entropy", max_depth=MAX_DEPTH, max_leaf_nodes=MAX_LEAF, min_impurity_decrease=IMPURITY),
    "RandomForestClassifier": RandomForestClassifier(criterion="entropy", max_depth=MAX_DEPTH, max_leaf_nodes=MAX_LEAF, min_impurity_decrease=IMPURITY),
    "XGBoostClassifier": XGBClassifier(max_depth=MAX_DEPTH, max_leaves=MAX_LEAF, learning_rate=LEARNING_RATE)
}

model_dirs = {
    "LogisticRegression": "models/Logistic_Regression/",
    "KNNClassifier": "models/Random_Forest/",
    "DecisionTreeClassifier": "models/KNN/",
    "RandomForestClassifier": "models/XGBoost/",
    "XGBoostClassifier": "models/Decision_Tree/"
}

In [107]:
# Implement KFold for cross validation
kf = KFold(n_splits=5, shuffle=True)

# Select X and y
X = data[['Year', 'District', 'generalized_loc', 'Community Area', 'Month', 'Primary Type', 'total_crimes_per_yr', 'monthly_crime_count']]
# Normalize X before training
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = data["Arrest"].to_numpy()

# Compare shape of both to see if they match
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Shape of X: (1089735, 8)
Shape of y: (1089735,)


Training each model for each fold

In [ ]:
for model_name, model in models.items():
    print(f"*** TRAINING {model_name} ***")
    folder = model_dirs[model_name]
    ensure_path(folder)

    # Arrays of metrics to store all folds
    acc_scores = []
    recall_scores = []
    precision_scores = []
    roc_auc_scores = []
    f1_scores = []

    
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        y_hat = model.predict(X_test)
        
        # Evaluate model metrics
        print("*** EVALUATING ***")
        accuracy = accuracy_score(y_test, y_hat)
        recall = recall_score(y_test, y_hat)
        precision = precision_score(y_test, y_hat)
        auc = roc_auc_score(y_test, y_hat)
        f1 = f1_score(y_test, y_hat)

        # Append to the array
        acc_scores.append(accuracy)
        recall_scores.append(recall)
        precision_scores.append(precision)
        roc_auc_scores.append(auc)
        f1_scores.append(f1)

        # Outputs the result
        print(f"Fold {i} ==>  Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, Area Under Curve: {auc},  F-1: {f1}")

        # Save the model 
        file_name = os.path.join(folder, f"{model_name.lower()}_fold_{i}.bin")
        joblib.dump(model, file_name)
        print(f"Saved {model_name} for fold {i} to {folder}")
        




Index(['ID', 'Case Number', 'Date', 'Primary Type', 'Description', 'Arrest',
       'Domestic', 'District', 'Ward', 'Community Area', 'Year', 'Month',
       'Latitude', 'Longitude', 'primary_type_count', 'total_crimes_per_yr',
       'monthly_crime_count', 'generalized_loc', 'Lat_round', 'Lon_round',
       'location_crime_count'],
      dtype='object')